# Web scraping stat Viego

In [118]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [119]:
url = "https://leagueoflegends.fandom.com/wiki/Viego/LoL"
url

'https://leagueoflegends.fandom.com/wiki/Viego/LoL'

*Statistic growth = base+ (perlvl) x (n-1) x (0.7025 + 0.0175 x (n-1))*

In [120]:
response = requests.get(url)
html = BeautifulSoup(response.text, "html.parser")

In [121]:
def findSpan(typeTag : str,  libTag : str):
    return html.find("span", {typeTag : libTag})

In [122]:
hp = findSpan("id","Health_Viego")
hp_lvl = findSpan("id","Health_Viego_lvl")
hp_list = [(float(hp.text) + float(hp_lvl.text) * (n-1) *(0.7025 +0.0175 * (n-1))) for n in range(1,19)]

In [123]:
armor =  findSpan("id","Armor_Viego")
armor_lvl = findSpan("id","Armor_Viego_lvl")
armor_list = [(float(armor.text) + float(armor_lvl.text) * (n-1) *(0.7025 +0.0175 * (n-1))) for n in range(1,19)]

In [124]:
rm = findSpan("id","MagicResist_Viego")
rm_lvl = findSpan("id","MagicResist_Viego_lvl")
rm_list = [(float(rm.text) + float(rm_lvl.text) * (n-1) *(0.7025 +0.0175 * (n-1))) for n in range(1,19)]

In [125]:
ad = findSpan("id","AttackDamage_Viego")
ad_lvl = findSpan("id","AttackDamage_Viego_lvl")
ad_list = [(float(ad.text) + float(ad_lvl.text) * (n-1) *(0.7025 +0.0175 * (n-1))) for n in range(1,19)]

In [126]:
attack_speed = html.findAll('div', {"style" : "width: calc(1 / 2 * 100%);"})
attack_speed = attack_speed[10]
attack_speed = float(re.sub(r'[^0-9.,]', '', attack_speed.text))
print(attack_speed)

0.658


In [127]:
bonus_as_lvl = html.findAll('div', {"style" : "width: calc(1 / 2 * 100%);"})
bonus_as_lvl = bonus_as_lvl[13]
bonus_as_lvl = float(re.sub(r'[^0-9.,]', '', bonus_as_lvl.text))
bonus_as_lvl /=100
print(bonus_as_lvl)
bonus_as_list = [ float(bonus_as_lvl) * (n-1) *(0.7025 +0.0175 * (n-1)) for n in range(1,19)]
print(bonus_as_list)

0.025
[0.0, 0.018, 0.036875000000000005, 0.05662500000000001, 0.07725, 0.09875, 0.12112500000000001, 0.14437500000000003, 0.1685, 0.19350000000000003, 0.21937500000000001, 0.24612500000000004, 0.27375000000000005, 0.30225, 0.33162500000000006, 0.36187500000000006, 0.393, 0.42500000000000004]


RATIO = 0.658
Formule total AS : *TOTAL = base + ratio*bonus

In [128]:
ratio = 0.658

In [129]:
as_list = [ attack_speed + ratio*bonus_as_list[n]  for n in range(0,18)]
print(as_list)

[0.658, 0.669844, 0.6822637500000001, 0.69525925, 0.7088305, 0.7229775, 0.7377002500000001, 0.75299875, 0.768873, 0.785323, 0.80234875, 0.8199502500000001, 0.8381275000000001, 0.8568805, 0.87620925, 0.89611375, 0.9165940000000001, 0.9376500000000001]


In [130]:
viegoStat = pd.DataFrame(index=range(1,19),columns=['hp','armor','magic_res','ad','as'])
viegoStat['hp'] = hp_list
viegoStat['armor'] = armor_list
viegoStat['magic_res'] = rm_list
viegoStat['ad'] = ad_list
viegoStat['as'] = bonus_as_list #Ici c'est le bonus d'attaque speed, il faut additionner les autres bonus
viegoStat

,hp,armor,magic_res,ad,as
1,630.000,34.000,32.00000,57.0000,0.000000
2,708.480,37.744,33.47600,59.5200,0.018000
3,790.775,41.670,35.02375,62.1625,0.036875
4,876.885,45.778,36.64325,64.9275,0.056625
5,966.810,50.068,38.33450,67.8150,0.077250
6,1060.550,54.540,40.09750,70.8250,0.098750
7,1158.105,59.194,41.93225,73.9575,0.121125
8,1259.475,64.030,43.83875,77.2125,0.144375
9,1364.660,69.048,45.81700,80.5900,0.168500
10,1473.660,74.248,47.86700,84.0900,0.193500


In [131]:
%run "../class_champion.ipynb"

In [132]:
viegoBase = Champion("viego",viegoStat)

In [133]:
print(viegoBase)

viego
          hp    armor  magic_res        ad        as
1    630.000   34.000   32.00000   57.0000  0.000000
2    708.480   37.744   33.47600   59.5200  0.018000
3    790.775   41.670   35.02375   62.1625  0.036875
4    876.885   45.778   36.64325   64.9275  0.056625
5    966.810   50.068   38.33450   67.8150  0.077250
6   1060.550   54.540   40.09750   70.8250  0.098750
7   1158.105   59.194   41.93225   73.9575  0.121125
8   1259.475   64.030   43.83875   77.2125  0.144375
9   1364.660   69.048   45.81700   80.5900  0.168500
10  1473.660   74.248   47.86700   84.0900  0.193500
11  1586.475   79.630   49.98875   87.7125  0.219375
12  1703.105   85.194   52.18225   91.4575  0.246125
13  1823.550   90.940   54.44750   95.3250  0.273750
14  1947.810   96.868   56.78450   99.3150  0.302250
15  2075.885  102.978   59.19325  103.4275  0.331625
16  2207.775  109.270   61.67375  107.6625  0.361875
17  2343.480  115.744   64.22600  112.0200  0.393000
18  2483.000  122.400   66.85000  116.50

In [135]:

def find_element_by_label(label, class_name):
    if html is None:
        return None

    balise_h3 = html.find('h3', class_='pi-data-label {}'.format(class_name), string=label)

    if balise_h3:
        balise_suivante = balise_h3.find_next_sibling()

        if balise_suivante:
            return balise_suivante.text.strip()


def get_cooldown_value():
    return find_element_by_label('COOLDOWN:', 'pi-secondary-font')

def Bonus_Damage():
    stats_div = html.find('div', {'style': 'padding: 0 7px'})

    if stats_div:
        stats_dl_list = stats_div.find_all('dl', class_='skill-tabs')
        stats = {}

        for stats_dl in stats_dl_list:
            dt = stats_dl.find('dt')
            dd = stats_dl.find('dd')

            if dt and dd:
                label = dt.text.strip().replace(':', '')
                value = dd.text.strip()
                stats[label] = value
        return stats

    return None

def format_to_list(chaine):
    nombres = []
    nombre_temporaire = ""
    est_dans_nombre = False

    for caractere in chaine:
        if caractere.isdigit() or caractere == '.':
            nombre_temporaire += caractere
            est_dans_nombre = True
        elif est_dans_nombre:
            nombres.append(float(nombre_temporaire))
            nombre_temporaire = ""
            est_dans_nombre = False

    if est_dans_nombre:
        nombres.append(float(nombre_temporaire))

    return nombres

'''
extrait le text sans balise
'''
def extract_text_without_tags(element):
    """
    Extract text content from HTML without any tags.
    """
    result = []
    for item in element.children:
        if isinstance(item, str):
            if item != "\n":
                result.append(item)
        else:
            result += html.extract_text_without_tags(item)

    return result

def get_divs_above_img(img_alt_text):
    img_tag = html.find('img', alt=img_alt_text)

    if img_tag:
        parent_div = img_tag.find_parent('div')
        divs_above = parent_div.find_all_previous('div', limit=1)

        return divs_above

    return None
def get_orange_and_blue_text(divs_above_img):
    # color #7A6DFF bleu et orange
    orange_and_blue_text = []

    for div in divs_above_img:
        orange_text = div.find_all('span', style="color:orange; white-space:normal",limit=2)
        blue_text = div.find_all('span', style="color: #7A6DFF; white-space:normal")

        for text in orange_text:
            orange_and_blue_text.append(text.get_text(strip=True))

        for text in blue_text:
            orange_and_blue_text.append(text.get_text(strip=True))

    return orange_and_blue_text

def find_physical_damage_table():
    physical_damage_span = html.find('span', {'class': 'template_lc'}, string='Physical Damage:')
    if physical_damage_span:
        table = physical_damage_span.find_previous('table')
        dd_elements = table.find('dd', class_='')
        dd_texts = [dd.get_text(strip=True) for dd in dd_elements]

    return dd_texts

def find_physical_damage_span():
    return html.find('span', {'class': 'template_lc'}, string='Physical Damage:')

# ancien main
cooldown_value = get_cooldown_value()
cooldown_value = format_to_list(cooldown_value)

print("Cooldown\n")
print(cooldown_value)
print("\n")
print("Bonus Damage Max/Min")
print(Bonus_Damage())
print("\n")
print("bonus damage 2")
img_alt_text = 'Blade of the Ruined King'
divs_above_img = get_divs_above_img(img_alt_text)
if divs_above_img:
    orange_and_blue_text = get_orange_and_blue_text(divs_above_img)
    print(orange_and_blue_text)
physical_damage_table = find_physical_damage_table()
if physical_damage_table:
    print("\nPhysical Damage Table:")
print(physical_damage_table)



Cooldown

[5.0, 4.5, 4.0, 3.5, 3.0]


Bonus Damage Max/Min
{'Bonus Physical Damage': "2 / 3 / 4 / 5 / 6% of target's current health", 'Minimum Bonus Damage': '10 / 15 / 20 / 25 / 30'}


bonus damage 2
['20% AD', '35% (+7%) AD', '(+ 15% AP)', '(+ 26.25% (+5.25%) AP)']

Physical Damage Table:
['15 / 30 / 45 / 60 / 75', '(+ 70% AD)', '', '×', '(1 + (100%\xa0+0%) critical strike chance)']
